In [ ]:
#Set your Imports
import os
import sys
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Add the project root to sys.path
notebook_dir = os.path.dirname(os.path.abspath('data.ipynb'))
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from datasharing.datasharing import DataSharingClient

In [ ]:
#Connect to datasharing client

client = DataSharingClient()


In [ ]:
#Connect to dataset
s3_uri = "s3://oceandatachallengebucket/public/francetax.parquet"
view_name = "francetax"
client.create_view(s3_uri, view_name)

In [ ]:
#Query a view or table and return a pandas dataframe
query = "SELECT COUNT(*) FROM francetax;"
result_df = client.query(query)
print(result_df)

In [ ]:
#Query a view or table and return a pandas dataframe
query = "SELECT COUNT(*) FROM francetax;"
result_df = client.query(query)
print(result_df)

In [ ]:
#Connect to dataset
s3_uri = "s3://datasharingexamples/nflverse/pbp_2023.parquet"
view_name = "nfl_pbp"
client.create_view(s3_uri, view_name)

In [ ]:
#Query a view or table and return a pandas dataframe
query = "SELECT COUNT(*) FROM nfl_pbp;"
result_df = client.query(query)
print(result_df)

In [ ]:
# Query a view or table and create a duckdb table
query = f"""
WITH redzone_drives AS (
    SELECT DISTINCT
        posteam,
        game_id,
        drive
    FROM 
        nfl_pbp
    WHERE 
        season = 2023
        AND week < 19
        AND yardline_100 <= 20
        AND play_type NOT IN ('no_play', 'extra_point', 'two_point_attempt')
),
drive_outcomes AS (
    SELECT 
        rd.posteam,
        rd.game_id,
        rd.drive,
        MAX(CASE WHEN touchdown = 1 THEN 1 ELSE 0 END) as drive_touchdown,
        MAX(CASE WHEN field_goal_attempt = 1 AND field_goal_result = 'made' THEN 1 ELSE 0 END) as drive_field_goal
    FROM 
        redzone_drives rd
    JOIN 
        nfl_pbp nfl
    ON 
        rd.posteam = nfl.posteam
        AND rd.game_id = nfl.game_id 
        AND rd.drive = nfl.drive
    WHERE 
        nfl.season = 2023
        AND nfl.week < 19
        AND nfl.yardline_100 <= 20
        AND nfl.play_type NOT IN ('no_play', 'extra_point', 'two_point_attempt')
    GROUP BY 
        rd.posteam, rd.game_id, rd.drive
)
SELECT 
    posteam,
    COUNT(*) as red_zone_trips,
    SUM(drive_touchdown) as touchdowns,
    SUM(drive_field_goal) as field_goals,
    ROUND(
        SUM(drive_touchdown) * 100.0 / NULLIF(COUNT(*), 0),
        2
    ) as touchdown_percentage
FROM 
    drive_outcomes
GROUP BY 
    posteam
ORDER BY 
    touchdown_percentage DESC;
"""
new_table_name = "red_zone_efficiency_2023_official"
count = client.query(query, new_table_name)

In [ ]:
#Query a view or table and return a pandas dataframe
query = "SELECT * FROM red_zone_efficiency_2023_official;"
result_df = client.query(query)
print(result_df)

In [ ]:
# View all tables and views in duckdb
tables = client.list_tables()
print(tables)

In [ ]:
# Define the S3 URI and local path
s3_uri = 's3://datasharingexamples/nflverse/pbp_2023.parquet'
local_path = '/home/christianocean/datasharing/data/pbp_2023.parquet'

# Download the dataset
client.download_dataset(s3_uri, local_path)


In [ ]:
#Export a duckdb table as csv or parquet
table_names = ["red_zone_efficiency_2023_official"]
output_dir = "/home/christianocean/datasharing/data"

# Export the tables to CSV format
client.export_tables(table_names, output_dir, "csv")#parquet for parquet export